<a href="https://colab.research.google.com/github/Pugianf/Big_Data_and_Public_Sector_III-Economic_Development/blob/main/Trablaho_Final_Big_Data_e_Setor_P%C3%BAblico_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Questão 1 - Biblioteca ipeadatapy e atualização statsmodel

In [1]:
# instalação ipeadatapy e atualização statsmodels

from IPython.display import clear_output

!pip uninstall statsmodels -y
!pip install statsmodels
!pip install ipeadatapy
!pip install pybacen
!pip install wbgapi
!pip install geopandas
!pip install mapclassify
!pip install geobr

clear_output() 

In [3]:
# importação de bibliotecas necessárias

# manipulação de dados

import pandas as pd
import numpy as np

# estatística e modelos

from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.api as sms

# gráficos

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick 
import matplotlib as mpl
import seaborn as sns

# mapas

import geopandas as gpd
import mapclassify
import geobr

# estilo dos gráficos

from matplotlib import style
style.use("ggplot")

# coleta de dados
import ipeadatapy
from pybacen.bacen import time_series 
import wbgapi as wb

## Questão 2 - Montando DataFrame com ipeadatapy

In [5]:
# verificando a base de dados

ipeadatapy.describe('PAN12_IPCAG12')

,Índice de Preços ao Consumidor Ampliado (IPCA)
Name,Índice de Preços ao Consumidor Ampliado (IPCA)
Code,PAN12_IPCAG12
Big Theme,Macroeconômico
Theme,Sinopse macroeconômica
Source,Instituto Brasileiro de Geografia e Estatístic...
Source acronym,IBGE/SNIPC
Comment,Índice Nacional de Preços ao Consumidor Amplo ...
Last update,2022-01-12T08:30:01.28-03:00
Frequency,Mensal
Measure,(% a.a.)


In [ ]:
# coletando taxa de inflação

dfIPCA = ipeadatapy.timeseries('PAN12_IPCAG12') 

dfIPCA

# conforme visto, tem sido apresentada inconsistência ao se solicitar a base de dados

In [7]:
!pip install seriesbr

from seriesbr import ipea

In [8]:
ipea.get_series("PAN12_IPCAG12")

ConnectTimeout: ignored

In [23]:
# utilizando csv baixado através da página http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=38391

dfIPCA = pd.read_csv("/content/drive/MyDrive/IDP/Dataset/PAN12_IPCAG12.CSV", sep=';', encoding='latin1')

In [28]:
dfIPCA

,Data,Índice de Preços ao Consumidor Ampliado (IPCA)
0,1980/01,115.70
1,1980/02,71.88
2,1980/03,102.10
3,1980/04,85.55
4,1980/05,94.55
...,...,...
499,2021/08,10.95
500,2021/09,14.84
501,2021/10,16.07
502,2021/11,12.02


In [31]:
# renomeando a coluna IPCA

dfIPCA.rename(columns={'Índice de Preços ao Consumidor Ampliado (IPCA)': 'IPCA'}, inplace=True)

dfIPCA.tail(5)

,Data,IPCA
499,2021/08,10.95
500,2021/09,14.84
501,2021/10,16.07
502,2021/11,12.02
503,2021/12,9.12


In [35]:
# setando Data como index e transformando a coluna Data para o formato correto:

dfIPCA['Data'] = pd.to_datetime(dfIPCA.Data)

dfIPCA.set_index('Data', inplace=True)

dfIPCA.tail(5)

,IPCA
Data,
2021-08-01,10.95
2021-09-01,14.84
2021-10-01,16.07
2021-11-01,12.02
2021-12-01,9.12
